### Import all what we need

In [ ]:
import os
import sys  
import requests
from six.moves import urllib
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

## Get Download Links

Download files of molecule C2H2.

In [ ]:
archive_url = "http://www.exomol.com/data/molecules/C2H2/12C2-1H2/aCeTY/"

Get the download links of states.bz2 files and trans.bz2 files from webset.

In [ ]:
def get_target_links():
    r = requests.get(archive_url)
    soup = BeautifulSoup(r.content, 'html5lib')
    links = soup.findAll('a')
    file_links = []
    
    #file_links = [archive_url + link['href'] for link in links if (link['href'].endswith('states.bz2') or link['href'].endswith('trans.bz2'))]
    for link in links:
        try:
            if((link['href'].endswith('states.bz2') or link['href'].endswith('trans.bz2'))):
            #if((link['href'].endswith('states.bz2'))):
                file_links.append("http://www.exomol.com" + link['href'])
        except KeyError:
            print('Keyerror, keep going!')
    for i in file_links:
        print(i, '\n')
    return file_links


In [37]:
target_links = get_target_links()

Keyerror, keep going!
Keyerror, keep going!
Keyerror, keep going!
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY.states.bz2 



## Download Files

Download states.bz2 files and trans.bz2 files with download links. Save these files into correspoding folders which names by download links and file names.

In [ ]:
def download_target_series(file_links):
    
    failed_list = [] 
    for link in tqdm(file_links):
        
        file_name = link.split('/')[-1]                
        print("Downloading file:%s" % file_name)
        print(link)

        # Make folders for save doanloaded files.
        full_name = "./data/" + archive_url.split('//')[-1]
        dir_name = "/".join(full_name.split('/')[:-1])
        if os.path.exists(dir_name):
            pass
        else:
            os.makedirs(dir_name, exist_ok=True)
        filename = os.path.join(full_name, file_name)
        
        try:
            r = requests.get(link, stream=True, verify=False)
        except Exception:
            failed_list.append(file_name.split('\\')[-1])
            print(' download failed. Go to download next one\n')
              
        # For compute the progess.
        total_size = int(r.headers['Content-Length'])
        temp_size = 0    
   
        # Download started.
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if chunk:
                    temp_size += len(chunk)
                    f.write(chunk)
                    f.flush()
                    done = int(50 * temp_size / total_size)
                    sys.stdout.write("\r[%s%s] %d%%" % ('█' * done, ' ' * (50 - done), 100 * temp_size / total_size))
                    sys.stdout.flush()
        
        
        print(" Downloaded!\n")

    print("All files downloaded!")

    print(failed_list) # Record which file is failed to download.

    return


In [13]:
download_target_series(target_links)

Keyerror, keep going!
Keyerror, keep going!
Keyerror, keep going!
http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY.states.bz2 



http://www.exomol.com/db/C2H2/12C2-1H2/aCeTY/12C2-1H2__aCeTY.states.bz2
[██████████████████████████████████████████████████] 100% Downloaded!


All files downloaded!
[]


## Decompress bz2 File

In [23]:
path = "./data/www.exomol.com/db/C2H2"

# Make a folder for saving decompressed files.
de_path = "./data/decompress/C2H2"
if os.path.exists(de_path):
    pass
else:
    os.makedirs(de_path, exist_ok=True)

    
for(dirpath,dirnames,files)in os.walk(path):
    count = 0
    for filename in files:
        filepath = os.path.join(dirpath, filename)
        de_filepath = os.path.join(de_path, filename.split('.')[0] + '_' + filename.split('.')[1] + '.inp')  
        count += 1
        print(count)
        with open(de_filepath, 'wb') as de_file, bz2.BZ2File(filepath, 'rb') as file:
            size = os.path.getsize(filepath)
            if (size < (float(1024) * 100)): 
                for data in iter(lambda: file.read(50 * 1024), b''):
                    de_file.write(data)
            else:
                for data in iter(lambda: file.read(100 * 1024), b''):
                    de_file.write(data)

                    
print("Finished decompression")


Finished decompression
